In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

!pip install pandarallel
from pandarallel import pandarallel as pl
pl.initialize(progress_bar=True, verbose=0)

# 設定使用環境
environment = 'kaggle'
#environment = 'colab'
#environment = 'client'

df_games = pd.DataFrame()
df_pffScouting = pd.DataFrame()
df_plays = pd.DataFrame()
df_players = pd.DataFrame()
df_tracking = pd.DataFrame()

/kaggle/input/nfl-big-data-bowl-2023/players.csv

/kaggle/input/nfl-big-data-bowl-2023/week6.csv

/kaggle/input/nfl-big-data-bowl-2023/week2.csv

/kaggle/input/nfl-big-data-bowl-2023/pffScoutingData.csv

/kaggle/input/nfl-big-data-bowl-2023/week3.csv

/kaggle/input/nfl-big-data-bowl-2023/week8.csv

/kaggle/input/nfl-big-data-bowl-2023/games.csv

/kaggle/input/nfl-big-data-bowl-2023/week5.csv

/kaggle/input/nfl-big-data-bowl-2023/week7.csv

/kaggle/input/nfl-big-data-bowl-2023/week1.csv

/kaggle/input/nfl-big-data-bowl-2023/week4.csv

/kaggle/input/nfl-big-data-bowl-2023/plays.csv


# Loading the example data

In [3]:
if (environment == 'kaggle'):
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
            if 'week5' in filename:
                weeks = os.path.join(dirname, filename)
            if 'players' in filename:
                players = os.path.join(dirname, filename)
            if 'pffScoutingData' in filename:
                pffScoutingData = os.path.join(dirname, filename)
            if 'plays' in filename:
                plays = os.path.join(dirname, filename)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

    df_pffScouting = pd.read_csv(pffScoutingData)
    df_plays = pd.read_csv(plays)
    df_players = pd.read_csv(players)
    df_tracking =  pd.read_csv(weeks)

elif(environment == 'colab') :
        raise NotImplementedError
    
elif(environment == 'client') :
    path = '../data/'
    
    df_games = pd.read_csv(path + 'games.csv')
    df_plays = pd.read_csv(path + 'plays.csv')
    df_players = pd.read_csv(path +'players.csv')
    df_pffScouting = pd.read_csv(path +'pffScoutingData.csv')
            
    
    for i in range(1,9,1):
        buffer = pd.read_csv(path + 'week' + str(i) + '.csv')
        df_tracking = pd.concat([df_tracking,buffer])
        df_tracking.append(buffer, ignore_index=True)
else:
    raise NotImplementedError
    

if (df_pffScouting.empty | df_plays.empty | df_players.empty | df_tracking.empty):
    raise Exception("檔案未讀取")

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0


In [7]:
df_gp = df_tracking[['gameId', 'playId']].drop_duplicates()
df_gp.reset_index(drop=True, inplace=True)
df_gp = df_gp[0:1]

In [8]:
df_gp

,gameId,playId
0,2021100700,95


# PreProcess Data

In [9]:
def Add_playerData_To_TrackingData(row ,playerDf):
    #apply to tracking data
    playerid = row['nflId']
    player = playerDf.loc[(playerDf['nflId'] == playerid)]
    #print(f'playerid :{playerid}')
    #print(f'player :{player}')
    try:
        nflId_displayName      = player.displayName.values[0]
        nflId_officialPosition = player.officialPosition.values[0]
        nflId_height           = player.height.values[0]
        nflId_weight           = player.weight.values[0]
    except:
        nflId_displayName=''
        nflId_officialPosition=''
        nflId_height=''
        nflId_weight=''
        
    row['nflId_displayName']      = nflId_displayName
    row['nflId_officialPosition'] = nflId_officialPosition
    row['nflId_height']           = nflId_height
    row['nflId_weight']           = nflId_weight
    return row
# Collect a record of the number of players in each position in every play.
# This will be used later to determine the maxiumum possible number of players
# for each position, which in turn determines how many features are needed
# to be collected (or zero-padded) at every position.
def get_position_counts(row,trackingDf):
    expr = (trackingDf['gameId'] == row['gameId']) & (trackingDf['playId'] == row['playId'])
    df = trackingDf.loc[expr]
    # 每frame參與play的位置數量應該相同
    # 在play中，所以只統計第一frame的位置數。
    vc = df.loc[df['frameId'] == 1, 'nflId_officialPosition'].value_counts()
    return pd.DataFrame(np.array([vc.values]), columns=vc.keys())

In [10]:
#把球員data塞進track data
player = df_tracking.parallel_apply(Add_playerData_To_TrackingData , axis=1,playerDf = df_players )

In [11]:
df_tracking = player

In [12]:
#統計每個position數量
pos_counts = df_gp.parallel_apply(get_position_counts, axis=1 , trackingDf = df_tracking)
pos_counts_max = pd.concat([df for df in pos_counts]).max().astype(int)
del pos_counts  # Mark temp object for deletion to free memory

WR    3
T     2
DE    2
FS    2
CB    2
dtype: int64

In [13]:
pos_counts_max.keys()

Index(['WR', 'T', 'DE', 'FS', 'CB', 'G', 'QB', 'DT', 'MLB', 'TE', 'SS', 'C',
       'NT', 'RB', 'OLB', ''],
      dtype='object')

In [14]:
filter = (df_tracking['gameId'] == 2021100700) &(df_tracking['playId'] == 95)\
         &(df_tracking['frameId'] == 1) & (df_tracking['nflId_officialPosition'] == 'T')

df_tracking[filter]

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,nflId_displayName,nflId_officialPosition,nflId_height,nflId_weight
0,2021100700,95,30869.0,1,2021-10-08T00:23:33.900,77.0,LA,right,22.93,33.04,0.02,0.02,0.00,102.26,129.51,None,Andrew Whitworth,T,6-7,330
282,2021100700,95,42400.0,1,2021-10-08T00:23:33.900,79.0,LA,right,23.26,26.67,0.01,0.01,0.01,103.56,5.00,None,Rob Havenstein,T,6-8,328


## 依據play id 將各個位置的frameId資料整合 當作一個Sequence

In [15]:
def format_features(row,tracking_data_source_df=None, all_positions=None ,feature_cols=None):
   
    #print(f'row:{row}')
    filter = (tracking_data_source_df['gameId'] == row['gameId']) \
             & (tracking_data_source_df['playId'] == row['playId'])
    
    ''' 取出相對應的play資料'''
    play_df = tracking_data_source_df.loc[filter]
    '''取得frames的集合並排序'''
    frames = sorted(play_df['frameId'].unique())
    #print(f'frames:{frames}')
    
    frame_data = [] #容器用以裝入處理好的frame data
    
    '''  Iterate through every frame in the play'''
    for fid in frames:
         #print(f'fid:{fid}')
         features = np.array([])
         '''Iterate每個位置，不一定在play每個位置都會有''' 
         for pos in all_positions:
                if pos!='':
                    pos_data = play_df.loc[(play_df['frameId'] == fid) &
                                   (play_df['nflId_officialPosition'] == pos),
                                   feature_cols].values
                    
                    pos_data =np.nan_to_num(np.hstack(pos_data))
                    #print(f'pos:{pos}')
                    #print(f'Typr:{type(pos_data)}')
                    #print(f'pos_data:{pos_data}')
                    features = np.hstack((features, pos_data))    
         #print(f'features:{features}')
         print(f'features len:{len(features)}')
         frame_data.append(features) 
         #break 
          
    return frame_data


def get_pass_results(row , plays_df):
    
    return plays_df.loc[(plays_df['gameId'] == row['gameId']) &
                        (plays_df['playId'] == row['playId']),
                        'passResult'].values[0]

In [16]:
# These are the column names to use as feature data.
feature_label_df = df_gp.copy()
feature_label_df['labels'] = feature_label_df.parallel_apply(get_pass_results, axis=1 , plays_df = df_plays)
feature_cols = ['nflId','x', 'y', 's', 'a', 'dis', 'o', 'dir','nflId_weight']
all_positions = sorted(pos_counts_max.keys())
feature_label_df['features']= df_gp.parallel_apply(format_features, axis=1, args=[ df_tracking,all_positions,feature_cols])

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198

features len:198


In [17]:
feature_label_df

,gameId,playId,labels,features
0,2021100700,95,C,"[[46180.0, 23.9, 29.93, 0.0, 0.0, 0.01, 60.14,..."


In [ ]:
#針對前處理的資料先排除label == R
feature_label_df = feature_label_df.drop(feature_label_df[feature_label_df.labels == 'R'].index)

#labels one hot encoding 
category_one_hot = {'C': [1.0, 0.0, 0.0, 0.0],'I': [0.0, 1.0, 0.0, 0.0],
                    'S': [0.0, 0.0, 1.0, 0.0],'IN': [0.0, 0.0, 0.0, 1.0]}
categories = feature_label_df['labels'].apply(lambda lbl: category_lookup[lbl])
labels_input = np.stack(categories)

#features
features_input = np.stack(feature_label_df['features'])

In [ ]:
from sklearn.model_selection import train_test_split
#切分訓練集(驗證集)、測試集
train_x, test_x, train_y, test_y = train_test_split(features_input, labels_input, test_size = 0.1)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.23)

# The Model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.losses import CategoricalCrossentropy
from keras.metrics import AUC

In [ ]:
frameNum = 64    #每一play的frame數量
featureNum = 184 #(每對11個選手 * 2 + 1 (ball)) * 8 特徵

model = Sequential()
model.add(LSTM(100, input_shape = (frameNum,featureNum)))
model.add(Dropout(0.2))
model.add(Dense(4, kernel_regularizer = 'l2')) #針對權重bias_regularizer
model.add(Activation('softmax'))
model.compile(loss = CategoricalCrossentropy(label_smoothing = 0.993),
              optimizer = 'adam', metrics = [AUC()])
model.summary()

In [ ]:
import matplotlib.pyplot as plt

# Trainging 
EPOCHS = 10
BATCH_SIZE = 100
history = model.fit(train_x, train_y,
                    validation_data = (val_x, val_y),
                    epochs = EPOCHS, batch_size = BATCH_SIZE)

In [ ]:
# Plot the loss and accuracy
fig = plt.figure(figsize=(15, 5)) #設定圖形大小

#左邊Loss圖
loss_plot = fig.add_subplot(1,2,1)
loss_plot.plot(history.history['loss'], color='blue', label='Train')
loss_plot.plot(history.history['val_loss'], color='darkorange', linestyle='dashed', label='Val')

loss_plot.title.set_text('Loss')
loss_plot.legend()

#右邊AUC
auc_plot = fig.add_subplot(1, 2, 2)

auc_hist = next(v for k, v in history.history.items() if k.startswith('auc')) #e.g.,auc_1 找aux前綴
auc_plot.plot(auc_hist, color='blue', label='Train')
val_auc_hist = next(v for k, v in history.history.items() if k.startswith('val_auc'))
auc_plot.plot(val_auc_hist, color='darkorange', linestyle='dashed', label='Val')

auc_plot.title.set_text('AUC')
auc_plot.legend()

In [ ]:
# 用測試集評估模型
score = model.evaluate(test_x, test_y, verbose = 2)
print('Loss:', score[0])
print('AUC:' , score[1])

## Trace Animate

In [ ]:
import pandas as pd
import numpy as np
import os
#import seaborn as sns
#from ipywidgets import interact, fixed
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import animation
from matplotlib.animation import FFMpegWriter

from datetime import datetime
import dateutil
from math import radians
from IPython.display import Video
import warnings
warnings.filterwarnings('ignore')

def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=55,
                          highlight_first_down_line=False,
                          yards_to_go=10,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        #plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
        #         color='yellow')
        
    if highlight_first_down_line:
        fl = hl + yards_to_go
        plt.plot([fl, fl], [0, 53.3], color='yellow')
        #plt.text(fl + 2, 50, '<- {}'.format(highlighted_name),
        #         color='yellow')
    return fig, ax

def calculate_dx_dy_arrow(x, y, angle, speed, multiplier):
    if angle <= 90:
        angle = angle
        dx = np.sin(radians(angle)) * multiplier * speed
        dy = np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    if angle > 90 and angle <= 180:
        angle = angle - 90
        dx = np.sin(radians(angle)) * multiplier * speed
        dy = -np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    if angle > 180 and angle <= 270:
        angle = angle - 180
        dx = -(np.sin(radians(angle)) * multiplier * speed)
        dy = -(np.cos(radians(angle)) * multiplier * speed)
        return dx, dy
    if angle > 270 and angle <= 360:
        angle = 360 - angle
        dx = -np.sin(radians(angle)) * multiplier * speed
        dy = np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    
        
def animate_player_movement(weekNumber, playId, gameId):
    path = './data/'
    
    if(environment == 'kaggle'):
        path = '/kaggle/input'
    elif (environment == 'client'):
        path = './data/'

    weekData = pd.read_csv(path + 'week' + str(weekNumber) + '.csv')
    playData = pd.read_csv(path + 'plays.csv')
    gameData = pd.read_csv(path + 'games.csv')
    game = gameData.loc[gameData['gameId'] == gameId]
    team_home = game['homeTeamAbbr'].iloc[0]
    team_visit = game['visitorTeamAbbr'].iloc[0]
    
    playHome = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "' + str(team_home) + '"')
    playAway = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "' + str(team_visit) + '"')
    playFootball = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "football"')
    
    playHome['time'] = playHome['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    playAway['time'] = playAway['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    playFootball['time'] = playFootball['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    # Need to replace NaN in the 'position' column with 'ball' to match the training data.
    #playFootball['position'].fillna('ball', inplace=True)

    maxTime = int(playAway['time'].unique().max())
    minTime = int(playAway['time'].unique().min())
    
    yardlineNumber = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['yardlineNumber'].item()
    yardsToGo = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['yardsToGo'].item()
    absoluteYardlineNumber = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['absoluteYardlineNumber'].item() - 10
    playDir = playHome.sample(1)['playDirection'].item()
    
    if (absoluteYardlineNumber > 50):
        yardlineNumber = 100 - yardlineNumber
    if (absoluteYardlineNumber <= 50):
        yardlineNumber = yardlineNumber
        
    if (playDir == 'left'):
        yardsToGo = -yardsToGo
    else:
        yardsToGo = yardsToGo
    
    fig, ax = create_football_field(highlight_line=True, highlight_line_number=yardlineNumber, highlight_first_down_line=True, yards_to_go=yardsToGo)
    playDesc = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['playDescription'].item()
    plt.title(f'Game # {gameId} Play # {playId} \n {playDesc}')
    
    def update_animation(time):
        patch = []
        
        homeX = playHome.query('time == ' + str(time))['x']
        homeY = playHome.query('time == ' + str(time))['y']
        homeNum = playHome.query('time == ' + str(time))['jerseyNumber']
        homeOrient = playHome.query('time == ' + str(time))['o']
        homeDir = playHome.query('time == ' + str(time))['dir']
        homeSpeed = playHome.query('time == ' + str(time))['s']
        patch.extend(plt.plot(homeX, homeY, 'o',c='gold', ms=20, mec='white'))
        
        # Home players' jersey number 
        for x, y, num in zip(homeX, homeY, homeNum):
            patch.append(plt.text(x, y, int(num), va='center', ha='center', color='black', size='medium'))
            
        # Home players' orientation
        for x, y, orient in zip(homeX, homeY, homeOrient):
            dx, dy = calculate_dx_dy_arrow(x, y, orient, 1, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='gold', width=0.5, shape='full'))
            
        # Home players' direction
        for x, y, direction, speed in zip(homeX, homeY, homeDir, homeSpeed):
            dx, dy = calculate_dx_dy_arrow(x, y, direction, speed, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='black', width=0.25, shape='full'))
        
        # Home players' location
        awayX = playAway.query('time == ' + str(time))['x']
        awayY = playAway.query('time == ' + str(time))['y']
        awayNum = playAway.query('time == ' + str(time))['jerseyNumber']
        awayOrient = playAway.query('time == ' + str(time))['o']
        awayDir = playAway.query('time == ' + str(time))['dir']
        awaySpeed = playAway.query('time == ' + str(time))['s']
        patch.extend(plt.plot(awayX, awayY, 'o',c='orangered', ms=20, mec='white'))
        
        # Away players' jersey number 
        for x, y, num in zip(awayX, awayY, awayNum):
            patch.append(plt.text(x, y, int(num), va='center', ha='center', color='white', size='medium'))
            
        # Away players' orientation
        for x, y, orient in zip(awayX, awayY, awayOrient):
            dx, dy = calculate_dx_dy_arrow(x, y, orient, 1, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='orangered', width=0.5, shape='full'))
        
        # Away players' direction
        for x, y, direction, speed in zip(awayX, awayY, awayDir, awaySpeed):
            dx, dy = calculate_dx_dy_arrow(x, y, direction, speed, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='black', width=0.25, shape='full'))
        
        # Away players' location
        footballX = playFootball.query('time == ' + str(time))['x']
        footballY = playFootball.query('time == ' + str(time))['y']
        patch.extend(plt.plot(footballX, footballY, 'o', c='black', ms=10, mec='white', data=playFootball.query('time == ' + str(time))['team']))
        
        # Display the pass outcome prediction in the title area
        features_df = pd.concat([playHome.loc[playHome['time'] <= time],
                                 playAway.loc[playAway['time'] <= time],
                                 playFootball.loc[playFootball['time'] <= time]])
        #prediction = get_prediction(features_df)
        #patch.append(plt.text(40, 55, f'Pass prediction: {prediction}', ha='left', color='red', size='x-large', weight='bold'))
        
        return patch
    
    ims = [[]]
    for time in tqdm(np.arange(minTime, maxTime+1)):
    #    for time in [minTime, minTime+1, minTime+2]:
        patch = update_animation(time)
        ims.append(patch)
        
    anim = animation.ArtistAnimation(fig, ims, repeat=False)
    
    return anim

## get animate video

In [ ]:
# video create
gameID = 2021091600
playID = 65
weekNum = 2
video = animate_player_movement(weekNum,playID,gameID)
video.save(path + 'video.mp4')